In [0]:
%fs
ls /Volumes/sample_catalog/default/db_catalog

In [0]:
# /Volumes/sample_catalog/default/db_catalog 

df = spark.read.csv("/Volumes/sample_catalog/default/db_catalog/customer.csv", header=True, inferSchema=True)
#df.show()
display(df)

In [0]:
df.printSchema()